In [118]:
# selenium 3
from selenium import webdriver
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.chrome.options import Options as FirefoxOptions
# firefox_options = FirefoxOptions()
# firefox_options.add_argument("--headless")
import pandas as pd
import requests
import random

In [143]:
driver = webdriver.Firefox(executable_path=GeckoDriverManager().install())

In [120]:
articles = pd.read_csv('/Users/antonhesse/Desktop/Anton/Education/UMN/Lab and Research/HSPL/CPET_scoping_review/data/cpet_articles/unpaywall/unpaywall_info.csv')
ovid_ca_articles = articles[(articles['is_oa'] == False) & (articles['publisher'] == 'Ovid Technologies (Wolters Kluwer Health)')].reset_index(drop=True)

In [121]:
n = random.randint(0, ovid_ca_articles.shape[0])
doi_url = ovid_ca_articles.loc[n, 'doi_url']

In [122]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:102.0) Gecko/20100101 Firefox/102.0'}
r = requests.get(doi_url, headers=headers)
r.url

'https://journals.lww.com/acsm-msse/Abstract/1994/01000/Comparison_of_cardiopulmonary_responses_to_forward.15.aspx'

In [144]:
driver.get(r.url)
driver.implicitly_wait(5) # hopefully let's JS load correctly

In [134]:
# pdf_download_icon = driver.find_element_by_class_name('icon-pdf')
# pdf_download_icon.get_attribute('innerHTML')

In [155]:
outer_pdf_button = driver.find_element_by_class_name('icon-pdf').find_element_by_xpath("../../..")
inner_pdf_icon = outer_pdf_button.find_element_by_xpath("//*[text()='PDF']")
# also will need to add text to find EPUBs if PDF is not available
pdf_download_button = inner_pdf_icon.find_element_by_xpath('..')
outer_pdf_button.click()

In [156]:
pdf_download_button.click()

In [157]:
# switch to new tab
p = driver.current_window_handle
chwd = driver.window_handles
for w in chwd:
    if(w != p):
        driver.switch_to.window(w)

In [133]:
# try: inner_pdf_button = outer_pdf_button.find_element_by_xpath("//*[text()='PDF']")

# except NoSuchElementException as e:
#     print(e)
#     try: inner_epub_button = outer_pdf_button.find_element_by_xpath("//*[text()='epub']")

In [158]:
pdf_resp = requests.get(url = driver.current_url, headers = headers)
pdf_resp.status_code

200

In [159]:
pdf_resp = requests.get(url = driver.current_url, headers = headers)
if pdf_resp.status_code == 200:
    doi = ovid_ca_articles.loc[n, 'doi']
    doi_suffix = str(doi.split('/', 1)[1:]).strip("[']")

    folder = '/Users/antonhesse/Desktop/Anton/Education/UMN/Lab and Research/HSPL/CPET_scoping_review/data/cpet_articles/full_texts/pdfs/ovid_non_oa/'
    filename = folder + str(doi_suffix)+'.pdf'
    
    with open(filename, mode = 'wb') as f:
        f.write(pdf_resp.content)

In [96]:
driver.current_url

'https://pdfs.journals.lww.com/nsca-jscr/2009/09000/Anthropometric_and_Metabolic_Determinants_of.31.pdf?token=method|ExpireAbsolute;source|Journals;ttl|1658347218237;payload|mY8D3u1TCCsNvP5E421JYK6N6XICDamxByyYpaNzk7FKjTaa1Yz22MivkHZqjGP4kdS2v0J76WGAnHACH69s21Csk0OpQi3YbjEMdSoz2UhVybFqQxA7lKwSUlA502zQZr96TQRwhVlocEp/sJ586aVbcBFlltKNKo+tbuMfL73hiPqJliudqs17cHeLcLbV/CqjlP3IO0jGHlHQtJWcICDdAyGJMnpi6RlbEJaRheGeh5z5uvqz3FLHgPKVXJzdFnRIX3TWX0v40bmDZD8Xeo1WUuSkjtVIabV2xNydipM=;hash|+eVA4bxNgR9BqVArLnxtvw=='

In [137]:
pdf_resp.status_code

200

In [114]:
doi = ovid_ca_articles.loc[n, 'doi']
doi_suffix = str(doi.split('/', 1)[1:]).strip("[']")

folder = '/Users/antonhesse/Desktop/Anton/Education/UMN/Lab and Research/HSPL/CPET_scoping_review/data/cpet_articles/full_texts/pdfs/ovid_non_oa/'
filename = folder + str(doi_suffix)+'.pdf'
filename

'/Users/antonhesse/Desktop/Anton/Education/UMN/Lab and Research/HSPL/CPET_scoping_review/data/cpet_articles/full_texts/pdfs/ovid_non_oa/jsc.0b013e3181b3dc7e.pdf'

In [115]:
with open(filename, mode = 'wb') as f:
    f.write(pdf_resp.content)

In [141]:
driver.close()

In [160]:
driver.quit()